# Importing Necessary Libraries

In [1]:
import numpy as np
import time
from sklearn.metrics import accuracy_score
import scipy.optimize as optim
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pandas as pd
import math
from tabulate import tabulate
from sklearn.model_selection import KFold, cross_val_score
from numpy import mean, std
from sklearn.preprocessing import StandardScaler
from random import seed, shuffle
import os,sys
import traceback
import numpy as np
from random import seed, shuffle
from collections import defaultdict
from copy import deepcopy
from cvxpy import *
import utils as ut
import cvxpy as cp
import dccp
from dccp.problem import is_dccp
from sklearn.metrics import accuracy_score

In [2]:
df_raw =pd.read_csv('https://github.com/propublica/compas-analysis/raw/master/compas-scores-two-years.csv')
#df_raw.columns

In [3]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7214 entries, 0 to 7213
Data columns (total 53 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       7214 non-null   int64  
 1   name                     7214 non-null   object 
 2   first                    7214 non-null   object 
 3   last                     7214 non-null   object 
 4   compas_screening_date    7214 non-null   object 
 5   sex                      7214 non-null   object 
 6   dob                      7214 non-null   object 
 7   age                      7214 non-null   int64  
 8   age_cat                  7214 non-null   object 
 9   race                     7214 non-null   object 
 10  juv_fel_count            7214 non-null   int64  
 11  decile_score             7214 non-null   int64  
 12  juv_misd_count           7214 non-null   int64  
 13  juv_other_count          7214 non-null   int64  
 14  priors_count            

In [4]:
df_raw.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
3,5,marcu brown,marcu,brown,2013-01-13,Male,1993-01-21,23,Less than 25,African-American,...,6,Medium,2013-01-13,NaN,NaN,1,0,1174,0,0
4,6,bouthy pierrelouis,bouthy,pierrelouis,2013-03-26,Male,1973-01-22,43,25 - 45,Other,...,1,Low,2013-03-26,NaN,NaN,2,0,1102,0,0


# Data Preprocessing

In [5]:
#calculate days in jail for each individual
df_raw['day_stayed'] =(pd.to_datetime(df_raw["c_jail_out"]) - pd.to_datetime(df_raw["c_jail_in"])).astype('timedelta64[h]') / 24

In [6]:
df_raw.corrwith(df_raw["two_year_recid"])

/var/folders/2j/xl2qzfzd781b4n_vgf720t2h0000gn/T/ipykernel_8063/1363567776.py:1: FutureWarning: The default value of numeric_only in DataFrame.corrwith is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  df_raw.corrwith(df_raw["two_year_recid"])


id                         0.016355
age                       -0.190323
juv_fel_count              0.092546
decile_score               0.350703
juv_misd_count             0.109310
juv_other_count            0.108004
priors_count               0.275117
days_b_screening_arrest    0.024481
c_days_from_compas        -0.067530
is_recid                   0.940544
r_days_from_arrest         0.043244
violent_recid                   NaN
is_violent_recid           0.356476
decile_score.1             0.350703
v_decile_score             0.295895
priors_count.1             0.275117
start                      0.080000
end                       -0.770794
event                      0.785957
two_year_recid             1.000000
day_stayed                 0.118377
dtype: float64

In [7]:
#drop irrelevant rows
#race != african_american or Caucasian
#days_stayed greater than 0
idx =np.where(((df_raw['race']=="African-American") |(df_raw['race']=="Caucasian"))
        & ((df_raw['sex']=="Male") |(df_raw['sex']=="Female"))
        & ((df_raw['day_stayed'] > 0)))

df =df_raw.loc[idx]

In [8]:
# dummy variables for race (Caucasian vs. African American) and sex (Male vs. Female)
df['sex'].replace(['Male','Female'],[1,0], inplace = True)
df['race'].replace(['Caucasian','African-American'],[1,0], inplace = True)

In [9]:
df.head()

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid,day_stayed
1,3,kevon dixon,kevon,dixon,2013-01-27,1,1982-01-22,34,25 - 45,0,...,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1,10.041667
2,4,ed philo,ed,philo,2013-04-14,1,1991-05-14,24,Less than 25,0,...,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1,1.083333
6,8,edward riddle,edward,riddle,2014-02-19,1,1974-07-23,41,25 - 45,1,...,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1,6.291667
8,10,elizabeth thieme,elizabeth,thieme,2014-03-16,0,1976-06-03,39,25 - 45,1,...,Low,2014-03-16,2014-03-15,2014-03-18,0,2,747,0,0,2.916667
9,13,bo bradac,bo,bradac,2013-11-04,1,1994-06-10,21,Less than 25,1,...,Medium,2013-11-04,2015-01-06,2015-01-07,1,0,428,1,1,0.958333


**Feature Selection**

Based on the correlation scores and considering fairness and relevance:

- Age 

- Criminal History Variables such as juv_fel_count, juv_misd_count, juv_other_count, and priors_count, which are all positively correlated with recidivism.

- Days Stayed 

- c_cha'rge_degree: Represents the charge degree (felony, misdemeanor) and can be a significant predictor for recidivism.

- is_recid and is_violent_recid: Both are directly related to recidivism. is_recid indicates if a person was re-arrested, but it has a correlation score of 0.9 with the target variable which can lead to data leakage. is_violent_recid indicates if the re-arrest was for a violent crime. It can be strong predictor.  


In [10]:
df['c_charge_degree'] =df['c_charge_degree'].astype('category').cat.codes


#Feature Selection and setting the target variable
X =df[['age', 'juv_fel_count', 'juv_misd_count', 'juv_other_count', 'priors_count', 'day_stayed', 'is_violent_recid', 'c_charge_degree' , 'sex']]
X

,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,day_stayed,is_violent_recid,c_charge_degree,sex
1,34,0,0,0,0,10.041667,1,0,1
2,24,0,0,1,4,1.083333,0,0,1
6,41,0,0,0,14,6.291667,0,0,1
8,39,0,0,0,0,2.916667,0,1,0
9,21,0,0,0,1,0.958333,1,0,1
...,...,...,...,...,...,...,...,...,...
7207,30,0,0,0,0,0.916667,0,1,1
7208,20,0,0,0,0,0.833333,0,0,1
7209,23,0,0,0,0,1.875000,0,0,1
7210,23,0,0,0,0,1.833333,0,0,1


In [11]:
#convert label to 1 and -1 since the implementation uses convex optimization techniques 
y = df['two_year_recid']
y[y==0] = -1 
Z = df['race']

/var/folders/2j/xl2qzfzd781b4n_vgf720t2h0000gn/T/ipykernel_8063/2000852408.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y[y==0] = -1


**Standarization**
Using StandardScaler() works better with constraints that involve operations like dot problems and sums.Standardizing features to have a mean of 0 and a standard deviation of 1 

In [12]:
#standarization
scaler =StandardScaler()
X =scaler.fit_transform(X)

Setting random_state = 6 ensures that the split is reproducible. 

In [13]:
#Splitting the Data into training, validating and testing sets
X_train, X_test, y_train, y_test, Z_train, Z_test =train_test_split(X, y, Z, test_size=0.3, random_state=6)

In [14]:
Z_train == 0

2821     True
6781     True
2844    False
4267     True
5076    False
        ...  
423      True
5985     True
2551     True
288      True
3525     True
Name: race, Length: 3989, dtype: bool

Helps in segregating the data based on the sensitive attribute, making the application of constraints separately for different groups.

# Model Training 

# Adding Constraints to Enforce Fairness in the Model

Includes the following steps:

- Setting up the model with maximum iterations for DCCP optimization and defining the shape of the model parameters (weights and bias).


- Defining the logistic regression loss function. This function quantifies the difference between the predicted values and actual labels.


- Calculating the proportions of the sensitive attribute in the data to use in fairness constraints.


- Creating fairness constraints. These constraints aim to reduce disparity in prediction errors (false positives and negatives) between different demographic groups.


- Setting up the optimization problem with the logistic loss function and added fairness constraints.


- Solving the optimization problem using the DCCP method. This method can handle the non-convex fairness constraints.


- Extracting the learned weights and bias from the model after optimization.

In [15]:
def train_model(x, y, z, c):
    maxiIters =100 
    maxIterdccp =50  

    numPoints, numFeatures = x.shape


    w =cp.Variable(x.shape[1])
    bias = cp.Variable()

    
    loss =cp.sum(cp.pos(1 - cp.multiply(y, x @ w + bias)))
    
    constraints = []

    
    totalSamples =len(z)
    s1 =np.sum(z)
    s0 =totalSamples - s1

    group0Indices =z==0
    group1Indices =z==1

    g00 =(-s1/totalSamples)*cp.sum(cp.minimum(0, cp.multiply(y[group0Indices], (x[group0Indices] @ w + bias))))
    g01 =(s0/totalSamples)*cp.sum(cp.minimum(0, cp.multiply(y[group1Indices], (x[group1Indices] @ w + bias))))

    constraints.append( g00 <= -g01 + c)
    constraints.append( g00 >= -g01 - c)

    #optimization problem with both the loss function and fairness constraints
    problem = cp.Problem(cp.Minimize(loss), constraints)
    
    try:
        problem.solve(method='dccp')
    except:
        traceback.print_exc()
        sys.stdout.flush()
        sys.exit(1)

    w =np.array(w.value).flatten() 
    
    return w, bias.value

# Calculating FPR and FNR

In [16]:
def calculate_group_metrics(y_true, y_pred, group_indicator):
    metrics = {}
    for group in np.unique(group_indicator):
        # Indices for the current group
        group_indices = group_indicator == group

        # Metrics calculation for the group
        tp = np.sum((y_true[group_indices] == 1) & (y_pred[group_indices] == 1))
        tn = np.sum((y_true[group_indices] == -1) & (y_pred[group_indices] == -1))
        fp = np.sum((y_true[group_indices] == -1) & (y_pred[group_indices] == 1))
        fn = np.sum((y_true[group_indices] == 1) & (y_pred[group_indices] == -1))

        # Calculate FPR and FNR
        fpr = fp / (fp + tn) if (fp + tn) > 0 else 0
        fnr = fn / (fn + tp) if (fn + tp) > 0 else 0

        # Store the metrics
        metrics[group] = {'FPR': fpr, 'FNR': fnr}

    return metrics


# Classification

In [17]:
def classification(c):
    
    w,b =train_model(X_train, y_train, Z_train, c)

    dbTrain =np.dot(X_train, w) + b
    yHatTrain = np.sign(dbTrain)

    dbTest =np.dot(X_test, w) + b
    yHatTest = np.sign(dbTest)
    
    # Convert to numpy arrays for easier handling
    y_train_np = np.array(y_train)
    y_test_np = np.array(y_test)
    Z_train_np = np.array(Z_train)
    Z_test_np = np.array(Z_test)
    
    yHatTrain_np = np.sign(dbTrain).astype(int)
    yHatTest_np = np.sign(dbTest).astype(int)


    # Calculate metrics for both groups
    train_metrics = calculate_group_metrics(y_train_np, yHatTrain_np, Z_train_np)
    test_metrics = calculate_group_metrics(y_test_np, yHatTest_np, Z_test_np)

    group0Indices =Z_train==0
    group1Indices =Z_train==1

    correctTrain =(yHatTrain == y_train).astype(int) 
    trainScore =(np.sum(correctTrain))/(len(correctTrain))

    acc_difTrain =np.abs(np.sum(correctTrain[group0Indices])/len(group0Indices) - np.sum(correctTrain[group1Indices])/len(group1Indices))
    
    group0Indices =Z_test==0
    group1Indices =Z_test==1
    
    correctTest =(yHatTest == y_test).astype(int)
    print(np.sum(correctTest))
    testScore =(np.sum(correctTest))/(len(correctTest))

    acc_difTest =np.abs(np.sum(correctTest[group0Indices])/len(group0Indices) - np.sum(correctTest[group1Indices])/len(group1Indices))

    
    
    return trainScore, testScore, acc_difTrain, acc_difTest, train_metrics, test_metrics

# Evaluation

The evaluation procedure involves computing accuracy, calibration differences, False Positive Rate (FPR), and False Negative Rate (FNR) for different groups. 

- Accuracy measures the overall effectiveness of the model. 
- calibration measures differences
- FPR/FNR assess fairness, specifically how model predictions differ across Caucasian and African-American groups.


**Justification**

* The procedure is reasonable as it includes both performance and fairness metrics, essential for a comprehensive understanding of the model's behavior in a societal context.

* The measures are intuitive. Accuracy is a standard metric for model performance. FPR and FNR provide insights into potential biases, offering a clear understanding of the model's fairness across different groups, which is crucial in the context of machine learning fairness. 

In [18]:
c = 0.05

In [19]:
train_acc, test_acc, acc_dif_train, acc_dif_test, train_metrics, test_metrics  = classification(c)
print("Training accuracy:", train_acc)
print("Test accuracy:", test_acc)
print("Calibration train:", acc_dif_train)
print("Calibration test:", acc_dif_test)

/Users/mansi/anaconda3/lib/python3.11/site-packages/cvxpy/reductions/solvers/solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)


1205
Training accuracy: 0.6898972173477061
Test accuracy: 0.7046783625730995
Calibration train: 0.12985710704437203
Calibration test: 0.11637426900584796


**Training Metrics**

In [25]:
print(f"Training FPR for Caucasian group: {train_metrics[1]['FPR']}")
print(f"Training FPR for African-American group: {train_metrics[0]['FPR']}")

Training FPR for Caucasian group: 0.07626208378088077
Training FPR for African-American group: 0.152753108348135


In [26]:
print(f"Training FNR for Caucasian group: {train_metrics[1]['FNR']}")
print(f"Training FNR for African-American group: {train_metrics[0]['FNR']}")

Training FNR for Caucasian group: 0.6100151745068285
Training FNR for African-American group: 0.46504320502749413


**Testing Metrics**

In [27]:
print(f"Test FPR for Caucasian group: {test_metrics[1]['FPR']}")
print(f"Test FPR for African-American group: {test_metrics[0]['FPR']}")

Test FPR for Caucasian group: 0.05660377358490566
Test FPR for African-American group: 0.1483050847457627


In [28]:
print(f"Test FNR for Caucasian group: {test_metrics[1]['FNR']}")
print(f"Test FNR for African-American group: {test_metrics[0]['FNR']}")

Test FNR for Caucasian group: 0.6156716417910447
Test FNR for African-American group: 0.45054945054945056
